# Posit Arithmetic Tests

![image](7_1_posits.png)

#### Test 1: unchecked positive +
a + b = c

In [29]:
from pyposit import posit

def basic_addition(a_: float = 8.0, b_: float = 4.0, es = 1):
    print(f"find {a_} + {b_}")
    a = posit(es, "0110100")
    a.from_float(a_, 7, es)
    b = posit(es, "1010000")
    b.from_float(b_, 7, es)
    print(f"A: {a}, B: {b}")

    ## extract signs
    a_s = a.sign_i()
    b_s = b.sign_i()

    ## extract fraction numeric values
    a_f_s = a.frac_str()
    b_f_s = b.frac_str()
    a_f = 0
    b_f = 0
    if a.frac_len():
        f = 2**(-a.frac_len())*int(a.frac_str(), 2)
    if b.frac_len():
        f = 2**(-b.frac_len())*int(b.frac_str(), 2)

    ## extract exponent numeric values
    a_e = 0
    b_e = 0
    if a.exp_len():
        a_e = int(a.exp_str(), 2)
    if b.exp_len():
        b_e = int(b.exp_str(), 2)

    ## extract regime numeric values
    if a.rbar_str() == "1":
        a_r = -1*a.regime_len()
    else:
        a_r = a.regime_len() - 1
    if b.rbar_str() == "1":
        b_r = -1*b.regime_len()
    else:
        b_r = b.regime_len() - 1

    print(f"A~ S:{a_s} R:{a_r} E:{a_e} F:{a_f}")
    print(f"B~ S:{b_s} R:{b_r} E:{b_e} F:{b_f}")

    ## Check which is bigger and shift to match
    ## if they are equal it doesn't really matter as shift will be 0

    sf = 2**(es) # es
    frac_smol = ""
    frac_big = ""
    e_out = -1 # pre-shift the p1
    r_out = 0
    
    # a >= b
    if (a_r > b_r) or \
       (a_r == b_r and a_e > b_e) or \
       (a_r == b_r and a_e == b_e and a_f >= b_f):
        ## a is the larger
        print("A is larger")
        exp_adj = sf*(a_r-b_r) + (a_e-b_e)
        # use its base exponents
        e_out += a_e
        r_out += a_r
        ## shift the smaller numbers fraction so that it matches 
        ## add the extra 1 thats hidden (we will do this for big aswell)
        frac_smol = "1" + b_f_s
        frac_smol = "0"*(exp_adj) + frac_smol
        frac_big = "1" + a_f_s
    else:
        ## b is the larger
        print("B is larger")
        exp_adj = sf*(b_r-a_r) + (b_e-a_e)
        e_out += b_e
        r_out += b_r
        ## shift the smaller numbers fraction so that it matches 
        ## add the extra 1 thats hidden (we will do this for big aswell)
        frac_smol = "1" + a_f_s
        frac_smol = "0"*(exp_adj) + frac_smol
        frac_big = "1" + b_f_s

    # extend big to match depth
    frac_big = frac_big + (len(frac_smol) - len(frac_big))*"0"

    print(f"big:   0.{frac_big}")
    print(f"small: 0.{frac_smol}")

    ## peform the fractional addition
    ## add and zero extend
    f_sum = bin(int(frac_big, 2) + int(frac_smol, 2))[2:]
    print(f"Sum: {frac_big} + {frac_smol} = {f_sum}")

    if len(f_sum) > len(frac_smol):
        ## overflow case
        #raise BaseException("Fraction Summation Overflow")
        e_out += 1
    elif len(f_sum) <= len(frac_smol):
        f_sum = (max(len(frac_smol), len(frac_big)) - len(f_sum))*"0" + f_sum

    print(f"Sum: 0.{f_sum}")

    ## normalise the sum (i.e. reshift until the first 1 is gone)
    c = f_sum.find("1") + 1
    f_sum = f_sum[c:]
    print(f"C Normalised by {c}: 1.{f_sum}")
    ## update the exponent with the normalised shift
    e_out = e_out + c
    ## recompare the regime and the exponent levels
    ## 2^e8 vs 2^2^es^r8
    print(f"Prenormal: R:{r_out} E:{e_out} F:0.{f_sum}")
    if (e_out > 2**es):
        e_out -= 2**es
        e_out += 1
    print(f"Posnormal: R:{r_out} E:{e_out} F:0.{f_sum}")

    ## we now have all the ideal required parts, convert to closest posit repr by available space
    n = 7
    if 1 + r_out >= n:
        print("Infinity")
    elif 1 + r_out + 1 == n:
        #  s r rbar 
        f_sum = "0"
        e8 = 0
    elif 1 + r_out + 1 + 1 == n:
        #  s r rbar e
        # no frac
        f_sum = "0"
    else:
        f_sum = f_sum[:n - (1 + r_out + 1 + 1) -1]

    f = 2**(-len(f_sum))*int(f_sum, 2)
    sff = 2**(2**(es))


    print(f"Poscat: R:{r_out} E:{e_out} F:{f}")
    print( (1 + f) * 2**(e_out) * sff**(r_out) )

basic_addition(0.25, 1.0)

    

find 0.25 + 1.0
A: 0010000, B: 0100000
A~ S:1 R:-1 E:0 F:0
B~ S:1 R:0 E:0 F:0
B is larger
big:   0.100000
small: 0.001000
Sum: 100000 + 001000 = 101000
Sum: 0.101000
C Normalised by 1: 1.01000
Prenormal: R:0 E:0 F:0.01000
Posnormal: R:0 E:0 F:0.01000
Poscat: R:0 E:0 F:0.25
1.25
